# \sigma_{a2}^{Sm} в зависимости от плотности теплоносителя, температуры теплоносителя, температуры топлива и концентрации борной кислоты

## 1. Загрузка данных

In [1]:
import pandas as pd

Загрузка данных

In [2]:
df = pd.read_excel('Input/Data.xlsx')
df.head(3)

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),3S_{tr1}(1),3S_{tr2}(2),\Sigma_{a1}(3),\Sigma_{a2}(4),nuS_{f1}(5),nuS_{f2}(6),\Sigma_{f1}(7),\Sigma_{f2}(8),\Sigma_{1->2}(9),k_{inf}(10),\sigma_{a2}^{Xe},\sigma_{a2}^{Sm}
0,0.7235,575.0,1027.0,600.0,0.661853,2.568298,0.009376,0.085584,0.006677,0.133250,0.002620,0.054750,0.017392,1.261041,1072970.0,35421.398
1,0.7750,448.0,1043.0,1500.0,0.679602,2.874687,0.009581,0.098339,0.006723,0.139256,0.002636,0.057218,0.018726,1.174265,1141000.0,36147.801
2,0.9125,370.5,1418.0,750.0,0.727336,3.668742,0.009908,0.102815,0.006844,0.153823,0.002681,0.063203,0.022463,1.249619,1308480.0,39214.000


In [3]:
names = ['\\rho(g/cm^3)',
 'T_c(K)',
 'T_f(K)',
 'c_b(ppm)',
 '\sigma_{a2}^{Sm}']

In [4]:
df = df[names]
df.head(3)

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),\sigma_{a2}^{Sm}
0,0.7235,575.0,1027.0,600.0,35421.398
1,0.7750,448.0,1043.0,1500.0,36147.801
2,0.9125,370.5,1418.0,750.0,39214.000


### 2 Нормализация признаков

In [5]:
features = ['\\rho(g/cm^3)',
 'T_c(K)',
 'T_f(K)',
 'c_b(ppm)']

In [6]:
df_features_norm = df.copy()

In [7]:
for i in range(len(features)):
    df_features_norm[features[i]] = (df[features[i]]-df[features[i]].mean())/df[features[i]].std()

In [8]:
df_features_norm

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),\sigma_{a2}^{Sm}
0,-0.316426,1.416398,-0.039094,-1.035019,35421.398
1,0.007922,-0.005506,-0.001946,0.006984,36147.801
2,0.873902,-0.873204,0.868693,-0.861352,39214.000
3,-0.858058,0.862191,-0.872586,0.875321,32457.400
4,-0.425067,-0.439355,0.433373,-1.295520,36604.801
...,...,...,...,...,...
252,-0.844527,-0.886762,-1.539169,-0.603564,34268.102
253,0.887433,0.848634,0.202111,1.133108,36142.898
254,0.021453,-0.019064,1.072750,-1.471900,38047.500
255,-1.710507,1.716332,-0.668529,0.264771,29368.699


In [9]:
df_features_norm.sample(frac=1) 

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),\sigma_{a2}^{Sm}
180,-0.113856,-1.618882,-1.131057,-1.119139,36086.398
28,-0.749810,-0.764742,-1.199075,1.635115,32388.900
46,0.386788,-1.035897,0.596617,-0.807081,37910.898
178,1.185113,-1.185033,-0.695737,-0.684971,39165.898
224,-1.602259,-0.127526,-1.212679,0.590398,30600.100
...,...,...,...,...,...
79,-1.426357,0.292764,0.895901,0.414016,31220.400
177,0.319134,1.418060,-1.566376,0.183366,35673.199
120,-1.263985,-1.605324,-0.355644,0.034121,31629.000
142,0.265010,-1.239264,1.644105,-1.281952,38111.898


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_features_norm[['\\rho(g/cm^3)', 'T_c(K)', 'T_f(K)', 'c_b(ppm)']],
                                                    df_features_norm[['\sigma_{a2}^{Sm}']],
                                                    test_size=0.33,
                                                    random_state=42)

# Нейросетевые модели

In [11]:
%%time

# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf

# Function to create model, required for KerasRegressor
def create_model(activation='relu', learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation=activation))
    model.add(Dense(1, kernel_initializer='normal'))
    
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = learn_rate, rho = 0.99, momentum = momentum, epsilon=1e-10)
    
    # Compile model
    model.compile(loss='mse', optimizer=optimizer)
    return model

X = X_train
Y = y_train

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 15, 20 ,25]
epochs = [700, 1000, 1300]
activation = ['tanh', 'sigmoid', 'linear']
learn_rate = [0.02, 0.04, 0.06, 0.08, 0.1, 0.3, 0.5, 0.7]
momentum = [0]

param_grid = dict(batch_size=batch_size,
                  epochs=epochs,
                  activation=activation,
                  learn_rate=learn_rate,
                  momentum=momentum
                 )
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=5)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -249218.525000 using {'activation': 'linear', 'batch_size': 25, 'epochs': 1000, 'learn_rate': 0.1, 'momentum': 0}
-1203070848.000000 (20601991.243017) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 700, 'learn_rate': 0.02, 'momentum': 0}
-1135459993.600000 (19977467.176589) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 700, 'learn_rate': 0.04, 'momentum': 0}
-1069872819.200000 (19445221.831180) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 700, 'learn_rate': 0.06, 'momentum': 0}
-1006316249.600000 (18760309.305978) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 700, 'learn_rate': 0.08, 'momentum': 0}
-944735692.800000 (18082668.340227) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 700, 'learn_rate': 0.1, 'momentum': 0}
-438505145.600000 (12115273.846248) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 700, 'learn_rate': 0.3, 'momentum': 0}
-129834880.000000 (6359122.995930) with: {'activation': 'tanh', 'batch_size': 10

In [12]:
'activation': 'linear', 'batch_size': 25, 'epochs': 1000, 'learn_rate': 0.1

SyntaxError: invalid syntax (<ipython-input-12-acf08cede729>, line 1)

## Модель 1

In [71]:
def create_model():

    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation='linear'))
    model.add(Dense(1, kernel_initializer='normal'))

    optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.1, rho = 0.001, momentum = 0, epsilon=1e-10)
    
    # Compile model
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [72]:
model = create_model()
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [73]:
# Обучение модели

EPOCHS = 1000
batch_size = 25

history = model.fit(
  X, Y,
  epochs=EPOCHS, batch_size = batch_size,verbose=0)

In [74]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail(10)

,loss,epoch
990,250215.140625,990
991,261438.531250,991
992,261798.234375,992
993,255804.390625,993
994,251459.343750,994
995,256711.531250,995
996,260217.609375,996
997,261017.859375,997
998,254321.843750,998
999,252100.390625,999


In [75]:
import statsmodels as statsmodels
import statsmodels.api as sm

RMSPE = statsmodels.tools.eval_measures.rmspe(y_test.values, model.predict(X_test))
RMSPE

array([0.18581445])

In [76]:
import statsmodels as statsmodels
import statsmodels.api as sm

RMSPE = statsmodels.tools.eval_measures.rmspe(y_train.values, model.predict(X_train))
RMSPE

array([0.13925269])